In [1]:
# install Requried Library
!pip install nltk
!pip install textblob
!pip install emoji
!pip install distance
!pip install fuzzywuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16257 sha256=d279491804eeaf1ad8997ddc28508fc67acc0f6a4bdd0815272835c0abe010cc
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [3]:
#Basic library
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# import preprocessing library
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import string
import emoji
from textblob import TextBlob
import distance
from fuzzywuzzy import fuzz
import re

import pickle
ngram_cv = pickle.load(open('ngram_cv.pkl','rb'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#Text PreProcessing
def preprocess(quest):
  # Lower case
  quest=str(quest).lower().strip()

  # Remove html tag
  quest=BeautifulSoup(quest)
  quest=quest.get_text()
  #print (quest)
  # replae certain special character with  their equivalents

  quest = quest.replace('%', ' percent')
  quest = quest.replace('$', ' dollar ')
  quest = quest.replace('₹', ' rupee ')
  quest = quest.replace('€', ' euro ')
  quest = quest.replace('@', ' at ')

  # The pattern '[math]' appears around 900 times in the whole dataset, we no meaning
  quest = quest.replace('[math]','')

  # Replacing some number with string equivalents (not perfect, can be done better to account for more cases)

  quest = quest.replace(',000,000,000', 'b ')
  quest = quest.replace(',000,000 ', 'm ')
  quest = quest.replace(',000', 'k')
  quest = re.sub(r'([0-9]+)000000000', r'\1b', quest)
  quest = re.sub(r'([0-9]+)00000', r'\1m', quest)
  quest = re.sub(r'([0-9]+)000', r'\1k',quest)

  # chatword treatment

  # Decontracting words
  # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
  # https://stackoverflow.com/a/19794953
  contractions = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "can not",
  "can't've": "can not have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "i will have",
  "i'm": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it would",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so as",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there would",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we would",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you would",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have",
  "'ve" : "have",
  "n't" : "not",
  "'re" : "are",
  "'ll" : "will"
  }

  quest_decontracted = []

  for word in quest.split():
    if word in contractions:
	    word = contractions[word]
    quest_decontracted.append(word)

  quest = ' '. join(quest_decontracted)

  #Remove punctuations
  for char in string.punctuation:
    quest=quest.replace(char,'')

  #Remove url # not needed
  #quest = re.compile(r'https?://\S+|www\.\S+').sub('', quest)

  # Remove stop word
  # will not remove stop word as we will use advance feature engineering
  # handling emojis # replace with it's meaning

  quest = emoji.demojize(quest)

  #spell check . No need
  #quest =TextBlob(quest).correct().string
  #print(quest)

  # Tokenization  # not needed

  # Semming   # will use lemmitization

  #lemmitization # not needed
  #lemmatizer = WordNetLemmatizer()
  #quest = ' '.join([lemmatizer.lemmatize(word) for word in quest.split()])

  return quest

In [6]:
# Feature Extraction
def common_words(q1,q2):

  w1 = set(map(lambda word: word.lower().strip(), q1.split(' ')))
  w2 = set(map(lambda word: word.lower().strip(), q2.split(' ')))
  return len(w1.intersection(w2)) # length of set intersection words, i.e common words betwen tow question

def total_words(q1,q2):

  w1 = set(map(lambda word: word.lower().strip(), q1.split(' ')))
  w2 = set(map(lambda word: word.lower().strip(), q2.split(' ')))
  return (len(w1) + len(w2))

#Baisc Feature Representation
def basic_features(q1,q2):

  query = [0.0]*7

  query[0]=len(q1)
  query[1]=len(q2)
  query[2]=len(q1.split(' '))
  query[3]=len(q2.split(' '))
  query[4]=common_words(q1,q2)
  query[5]=total_words(q1,q2)
  query[6]=round(common_words(q1,q2)/total_words(q1,q2),2)

  return query

#Token_advance_features
def token_features(q1,q2):

  STOP_WORDS = stopwords.words("english")
  token_features = [0.0]*8  # as we are returing 8 new featuresm to avoiding the missing of data we will define it in advance

  # convert the sentane into tokens:
  q1_tokens = q1.split()
  q2_tokens = q2.split()

  if len(q1_tokens) == 0 or len(q2_tokens) == 0 :
    return token_features

  # Get the non-stopword in Questions
  q1_words =set([word for word in q1_tokens if word not in STOP_WORDS])
  q2_words =set([word for word in q2_tokens if word not in STOP_WORDS])

  # Get te stopwrods in Question
  q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
  q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])

  #Get the common pwords from question pair
  common_word_count = len(q1_words.intersection(q2_words))

  #get the common stopwords from question pari
  common_stop_count = len(q1_stops.intersection(q2_stops))

  #get the common Token from Question pair
  common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))

  token_features[0] = common_word_count / (min(len(q1_words),len(q2_words)) + 1) # 1 for safe division
  token_features[1] = common_word_count / (max(len(q1_words),len(q2_words)) + 1) # 1 for safe division
  token_features[2] = common_stop_count / (min(len(q1_stops),len(q2_stops)) + 1) # 1 for safe division
  token_features[3] = common_stop_count / (min(len(q1_stops),len(q2_stops)) + 1) # 1 for safe division
  token_features[4] = common_token_count / (min(len(q1_tokens),len(q2_tokens)) + 1) # 1 for safe division
  token_features[5] = common_token_count / (max(len(q1_tokens),len(q2_tokens)) + 1) # 1 for safe division
  # Lastr word of the question is same or not
  token_features[6] = int(q1_tokens[-1]==q2_tokens[-1]) # last word equal
  # First word of the question is same or not
  token_features[7] = int(q1_tokens[0]==q2_tokens[0]) # first word equal

  return token_features

#Length_base_advance_features
def length_features (q1,q2):

  length_features = [0.0]*3

  #Sentance to tokens
  q1_tokens = q1.split()
  q2_tokens = q2.split()

  if len(q1_tokens) == 0 or len(q2_tokens) == 0 :
    return length_features

  # Abs length feature
  length_features[0]  = abs(len(q1_tokens) - len(q2_tokens))

  #Avg length features
  length_features[1] = np.mean(len(q1_tokens) + len(q2_tokens))

  strs = list(distance.lcsubstrings(q1, q2))
  if strs:
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
  else:
    length_features[2] = 0  # Assign a default value if strs is empty

  return length_features

#Fuzzybuzzy_advance_features
def fuzzy_features(q1,q2):

  fuzzy_features = [0.0]*4

  #fuzzy ratio
  fuzzy_features[0] = fuzz.QRatio(q1,q2)

  #fuzzy_partion_ratio
  fuzzy_features[1] = fuzz.partial_ratio(q1,q2)

  #fuzzy_token_sort_ratio
  fuzzy_features[2] = fuzz.token_sort_ratio(q1,q2)

  #token_set_ratio
  fuzzy_features[3] = fuzz.token_set_ratio(q1,q2)

  return fuzzy_features

In [7]:
def query_point_creator(q1,q2):
  query = []

  #preprocess
  q1 = preprocess(q1)
  q2 = preprocess(q2)

  # fetch basic features
  query.extend(basic_features(q1,q2))

  #fetch token features
  query.extend(token_features(q1,q2))

  #fetch length features
  query.extend(length_features(q1,q2))

  #fetch fuzzy features
  query.extend(fuzzy_features(q1,q2))

  #w2vfeatures for q1
  q1_ngram = ngram_cv.transform([q1]).toarray()

  #w2v features for q2
  q2_ngram = ngram_cv.transform([q2]).toarray()

  return np.hstack((np.array(query).reshape(1, 22), q1_ngram, q2_ngram))